# Synthetic data generator for asset networks
 Here we first create a list of assets; second we distribute randomly their ownership (fractional).

In [122]:
import random
import string

# code to generate a single random string
def get_random_string(length):
    # Only upper case
    letters = string.ascii_uppercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    # print random string
    #print(result_str)
    return result_str

# code to generate a list of random strings. Return approx "N=length" elements because identical ones are dropped
def gen_random_asset_list(length,name_length):
    my_asset_list = []
    for i in range(length):
        asset_name = get_random_string(name_length)
        my_asset_list.append(asset_name)
    #print(my_asset_list)
    
    #remove identical entries by passing the generated list into a set and back to a list.
    new_asset_list = list(set(my_asset_list))
    return new_asset_list


In [170]:
# random edge probability per node

import numpy as np

assetlist = gen_random_asset_list(10,2)
# generate a list of probabilities to a edge to exists from each node:
node_edge_prob = []
count = 0
for i in assetlist:
    node_edge_prob.append(np.random.uniform(0,0.5))
    print(i," ",node_edge_prob[count])
    count=count+1

MV   0.3259997565604753
IR   0.19260154566656057
KA   0.004226136022737692
XJ   0.2991100149355891
HS   0.2100499615618116
MC   0.3673614935926322
VQ   0.19500111130040026
YO   0.48581364054948944
GY   0.43298940997250834
GN   0.16383045184614448


In [171]:
#generating the edges
edges = []

count = 0
for i in assetlist:
    for j in assetlist:
        if np.random.uniform(0,1) < node_edge_prob[count] and i!=j:
            edges.append((i,j))
    count=count+1

print(edges)

[('MV', 'HS'), ('MV', 'VQ'), ('MV', 'GN'), ('XJ', 'MC'), ('XJ', 'GY'), ('XJ', 'GN'), ('MC', 'KA'), ('MC', 'XJ'), ('MC', 'HS'), ('MC', 'VQ'), ('MC', 'YO'), ('MC', 'GN'), ('VQ', 'GN'), ('YO', 'MV'), ('YO', 'IR'), ('YO', 'HS'), ('YO', 'VQ'), ('YO', 'GY'), ('GY', 'MV'), ('GY', 'IR'), ('GY', 'KA'), ('GY', 'MC'), ('GY', 'GN'), ('GN', 'VQ')]


In [190]:
#generate edges weigths: percentage over ownership

weights = [0]*len(edges)
print(weights)

#this will split the value in a given number of parts
def split_num_random(value,num_parts):
    dividing_points = []
    parts = []
    if num_parts == 1:
        parts.append(value)
        
    elif num_parts > 1:
        for i in range(0, num_parts-1):
            dividing_points.append(np.random.uniform(0,value))
        dividing_points.sort()
        #print(dividing_points)
        old_value = 0
        for i in range(len(dividing_points)):
            parts.append(dividing_points[i]-old_value)
            old_value = dividing_points[i]
        parts.append(value-old_value)
        #print(parts)
    return parts


#here edges and weights vectors must already be define
#this code associate divide the ownership into many parts, each corresponding to an in-vector to a node
def gen_edge_weights(node):
    # count number of connections for a node
    count = 0
    edges_index=[]
    
    for i in range(len(edges)):
        if edges[i][1] == node:
            edges_index.append(i)
            count=count+1
    
    #print(count)
    parts_local = split_num_random(1,count)
    #print(parts_local)
    
    count = 0
    for i in edges_index:
        #print(i)
        # weights are changed directly over the external variable
        weights[i]=parts_local[count]
        count=count+1

    return True

for i in assetlist:
    health=gen_edge_weights(i)
print(weights)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.7076738124889514, 0.7176222922431361, 0.1150366943981247, 0.7607923144767684, 0.14965219754815895, 0.18104865050233554, 0.4302738404545542, 1, 0.021179560793916363, 0.14554095647994247, 1, 0.10626099942720335, 0.05471799894001517, 0.7725168152260052, 0.7186313184494731, 0.2711466267171322, 0.016182567682019422, 0.850347802451841, 0.22748318477399476, 0.28136868155052686, 0.5697261595454458, 0.23920768552323157, 0.5429356567323212, 0.12065418359490199]


In [191]:
# plot the graph

import networkx as nx
#import matplotlib.pyplot as plt
from pyvis.network import Network

G=nx.DiGraph()
G.add_edges_from(edges)

net = Network(directed =True,notebook=True, cdn_resources='remote')
net.from_nx(G)
net.repulsion()
net.show("test.html")



test.html
